In [8]:
import sys
import random
'''
WRITE YOUR CODE BELOW.
'''
from numpy import zeros, float32
import numpy
#  pgmpy͏︆͏󠄃͏󠄌͏󠄍͏󠄂͏️͏󠄇͏︊͏︃
import pgmpy
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
#You are not allowed to use following set of modules from 'pgmpy' Library.͏︆͏󠄃͏󠄌͏󠄍͏󠄂͏️͏󠄇͏︊͏︃
#͏︆͏󠄃͏󠄌͏󠄍͏󠄂͏️͏󠄇͏︊͏︃
# pgmpy.sampling.*͏︆͏󠄃͏󠄌͏󠄍͏󠄂͏️͏󠄇͏︊͏︃
# pgmpy.factors.*͏︆͏󠄃͏󠄌͏󠄍͏󠄂͏️͏󠄇͏︊͏︃
# pgmpy.estimators.*͏︆͏󠄃͏󠄌͏󠄍͏󠄂͏️͏󠄇͏︊͏︃


def make_security_system_net():
    """Create a Bayes Net representation of the above security system problem. 
    Use the following as the name attribute: "H","C", "M","B", "Q", 'K",
    "D"'. (for the tests to work.)
    """
    BayesNet = BayesianModel()
    [BayesNet.add_node(i) for i in ["H","C", "M","B", "Q", "K", "D"]]
    BayesNet.add_edge("H","Q")
    BayesNet.add_edge("C","Q")
    BayesNet.add_edge("B","K")
    BayesNet.add_edge("M","K")
    BayesNet.add_edge("Q","D")
    BayesNet.add_edge("K","D")

    return BayesNet


def set_probability(bayes_net: BayesianModel):
    """Set probability distribution for each node in the security system.
    Use the following as the name attribute: "H","C", "M","B", "Q", 'K",
    "D"'. (for the tests to work.)
    """
    cpd_h = TabularCPD("H", 2, values= [[0.5], [0.5]])
    cpd_c = TabularCPD("C", 2, values= [[0.7], [0.3]])
    cpd_b = TabularCPD("B", 2, values= [[0.5], [0.5]])
    cpd_m = TabularCPD("M", 2, values= [[0.2], [0.8]])

    cpd_q = TabularCPD("Q", 2, values= [[0.95,	0.75,	0.45,	0.1], [0.05	,0.25	,0.55	,0.9]], evidence=["H","C"], evidence_card=[2, 2])
    cpd_k = TabularCPD("K", 2, values= [[0.25	,0.05	,0.99	,0.85], [0.75	,0.95	,0.01	,0.15]], evidence=["B","M"], evidence_card=[2, 2])
    
    cpd_d = TabularCPD("D", 2, values= [[0.98	,0.4	,0.65	,0.01], [0.02	,0.6	,0.35	,0.99]], evidence=["K","Q"], evidence_card=[2, 2])

    bayes_net.add_cpds(cpd_h, cpd_c ,cpd_b ,cpd_m ,cpd_q ,cpd_k, cpd_d)
    return bayes_net


def get_marginal_double0(bayes_net: BayesianModel):
    """Calculate the marginal probability that Double-0 gets compromised.
    """
    agent = VariableElimination(bayes_net)
    double0_prob = agent.query(variables= ['D'], joint=False)['D'].values[1]
    return double0_prob


def get_conditional_double0_given_no_contra(bayes_net):
    """Calculate the conditional probability that Double-0 gets compromised
    given Contra is shut down.
    """
    agent = VariableElimination(bayes_net)
    double0_prob = agent.query(variables= ['D'], evidence={"C":0},joint=False)['D'].values[1]
    return double0_prob


def get_conditional_double0_given_no_contra_and_bond_guarding(bayes_net):
    """Calculate the conditional probability that Double-0 gets compromised
    given Contra is shut down and Bond is reassigned to protect M.
    """
    agent = VariableElimination(bayes_net)
    double0_prob = agent.query(variables= ['D'], evidence={"C":0, 'B': 1},joint=False)['D'].values[1]
    return double0_prob


def get_game_network():
    """Create a Bayes Net representation of the game problem.
    Name the nodes as "A","B","C","AvB","BvC" and "CvA".  """
    BayesNet = BayesianModel()
    [BayesNet.add_node(i) for i in ["A","B","C","AvB","BvC","CvA"]]

    BayesNet.add_edge("A", "CvA")
    BayesNet.add_edge("A", "AvB")

    BayesNet.add_edge("B", "AvB")
    BayesNet.add_edge("B", "BvC")

    BayesNet.add_edge("C", "CvA")
    BayesNet.add_edge("C", "BvC")

    cpd_a = TabularCPD("A", 4, [[0.15] , [0.45], [0.30], [0.10]])
    cpd_b = TabularCPD("B", 4, [[0.15] , [0.45], [0.30], [0.10]])
    cpd_c = TabularCPD("C", 4, [[0.15] , [0.45], [0.30], [0.10]])

    

    a = build_tabels()
    values= [list(a.T[0]), list(a.T[1]), list(a.T[2])]
    cpd_ac = TabularCPD("CvA", 3, values , evidence=["A","C"], evidence_card=[4, 4])
    cpd_bc = TabularCPD("BvC", 3, values , evidence=["B","C"], evidence_card=[4, 4])
    cpd_ab = TabularCPD("AvB", 3, values , evidence=["A","B"], evidence_card=[4, 4])
    BayesNet.add_cpds(cpd_ab, cpd_ac, cpd_bc, cpd_a, cpd_b, cpd_c)


    return BayesNet

def build_tabels():
    skill_diff = {
    0:[0.10, 0.10, 0.80],
    1:[0.20, 0.60, 0.20],
    2:[0.15, 0.75, 0.10],
    3:[0.05, 0.90, 0.05],
    -1:[0.60, 0.20, 0.20],
    -2:[0.75, 0.15, 0.10],
    -3:[0.90, 0.05, 0.05]}
    a = zeros((16,3))
    b = zeros((4,4))
    col = 0
    for t_1 in [0, 1, 2, 3]: 
        for t_2 in [0, 1, 2, 3]:
            key = t_2 - t_1
            b[t_1][t_2] = key
            ab_0 = skill_diff[key][0]
            ab_1 = skill_diff[key][1]
            ab_2 = skill_diff[key][2]
            a[col] = [ab_0, ab_1, ab_2] 
            col += 1
    return a
 
def calculate_posterior(bayes_net):
    """Calculate the posterior distribution of the BvC match given that A won against B and tied C. 
    Return a list of probabilities corresponding to win, loss and tie likelihood."""
    agent = VariableElimination(bayes_net)
    posterior = agent.query(variables=["BvC"], joint= False, evidence= {'AvB': 0, 'CvA': 2})['BvC'].values
    return posterior 


def getA(bayes_net, evidence):
    """
    ***DO NOT POST OR SHARE THESE FUNCTIONS WITH ANYONE***
    Returns a distribution of probability of skill levels for team "A" given an evidence vector.
    Parameter: 
    : bayes net: Baysian Model Object
    : evidence: array of length 6 containing the skill levels for teams A, B, C in indices 0, 1, 2
    : and match outcome for AvB, BvC and CvA in indices 3, 4 and 5
    """
    A_cpd = bayes_net.get_cpds("A")      
    AvB_cpd = bayes_net.get_cpds("AvB")
    match_table = AvB_cpd.values
    team_table = A_cpd.values
    a = []
    normalizer = 0
    for i in range(4):
        normalizer += (team_table[i] * match_table[evidence[3]][i][evidence[1]] * 
                       match_table[evidence[5]][i][evidence[2]])
    for i in range(4):
        unnorm_prob = (team_table[i] * match_table[evidence[3]][i][evidence[1]] * 
                       match_table[evidence[5]][i][evidence[2]])
        a.append(unnorm_prob)
    return numpy.array(a)/normalizer

def getBvC(bayes_net, B, C):
    """
    ***DO NOT POST OR SHARE THESE FUNCTIONS WITH ANYONE***
    Returns a distribution of probability of match outcomes for "BvC" given the skill levels of B and C as evidence
    Parameter: 
    : bayes net: Baysian Model Object
    : B: int representing team B's skill level
    : C: int representing team C's skill level
    """
    BvC_cpd = bayes_net.get_cpds('BvC')
    match_table = BvC_cpd.values
    bvc = []
    for i in range(0, 3):
        bvc.append(match_table[i][B][C])
    return bvc   

def getB(bayes_net, evidence):
    """
    ***DO NOT POST OR SHARE THESE FUNCTIONS WITH ANYONE***
    Returns a distribution of probability of skill levels for team "B" given an evidence vector.
    Parameter: 
    : bayes net: Baysian Model Object
    : evidence: array of length 6 containing the skill levels for teams A, B, C in indices 0, 1, 2
    : and match outcome for AvB, BvC and CvA in indices 3, 4 and 5
    """
    B_cpd = bayes_net.get_cpds("B")      
    AvB_cpd = bayes_net.get_cpds("AvB")
    match_table = AvB_cpd.values
    team_table = B_cpd.values
    b = []
    normalizer = 0
    for i in range(4):
        normalizer += (team_table[i] * match_table[evidence[3]][evidence[0]][i] * 
                       match_table[evidence[4]][i][evidence[2]])
    for i in range(4):
        unnorm_prob = (team_table[i] * match_table[evidence[3]][evidence[0]][i] * 
                       match_table[evidence[4]][i][evidence[2]])
        b.append(unnorm_prob)
    return numpy.array(b)/normalizer

def getC(bayes_net, evidence):
    """
    ***DO NOT POST OR SHARE THESE FUNCTIONS WITH ANYONE***
    Returns a distribution of probability of skill levels for team "C" given an evidence vector.
    Parameter: 
    : bayes net: Baysian Model Object
    : evidence: array of length 6 containing the skill levels for teams A, B, C in indices 0, 1, 2
    : and match outcome for AvB, BvC and CvA in indices 3, 4 and 5
    """
    C_cpd = bayes_net.get_cpds("C")      
    AvB_cpd = bayes_net.get_cpds("AvB")
    match_table = AvB_cpd.values
    team_table = C_cpd.values
    c = []
    normalizer = 0
    for i in range(4):
        normalizer += (team_table[i] * match_table[evidence[5]][i][evidence[0]] * 
                       match_table[evidence[4]][evidence[1]][i])
    for i in range(4):
        unnorm_prob = (team_table[i] * match_table[evidence[5]][i][evidence[0]] * 
                       match_table[evidence[4]][evidence[1]][i])
        c.append(unnorm_prob)
    return numpy.array(c)/normalizer

def Gibbs_sampler(bayes_net: BayesianModel, initial_state):
    """Complete a single iteration of the Gibbs sampling algorithm 
    given a Bayesian network and an initial state value. 
    
    initial_state is a list of length 6 where: 
    index 0-2: represent skills of teams A,B,C (values lie in [0,3] inclusive)
    index 3-5: represent results of matches AvB, BvC, CvA (values lie in [0,2] inclusive)
    
    Returns the new state sampled from the probability distribution as a tuple of length 6.
    Return the sample as a tuple.    
    """
    if initial_state == None or len(initial_state) == 0 :

      initial_state = [random.choice(range(0,4)),random.choice(range(0,4)),random.choice(range(0,4)), 0, random.choice(range(0,4)), 2]

    possible_non_ev = ['A', 'B', 'C', 'BvC']
    targeted_var = random.choice(possible_non_ev)
    initial_state = list(initial_state)
    result = 0
    if  targeted_var == 'C':
        C = list(getC(bayes_net, initial_state))
        result = random.choices([0,1,2,3], C)[0]
        initial_state[2] = result

    elif targeted_var == 'A':
        A = list(getA(bayes_net, initial_state))
        result = random.choices([0,1,2,3], A)[0]
        initial_state[0] = result

    elif targeted_var == 'B':
        B = list(getB(bayes_net, initial_state))
        result = random.choices([0,1,2,3], B)[0]
        initial_state[1] = result

    elif targeted_var == 'BvC':
        BvC = getBvC(bayes_net, initial_state[1], initial_state[2])
        result = random.choices([0,1,2], BvC)[0]
        initial_state[4] = result
   
    
    sample = tuple(initial_state)
    return sample

def calculateMH(bayes_net, evidence):
    """
    ***DO NOT POST OR SHARE THESE FUNCTIONS WITH ANYONE***
    Returns the probability of a state.
    Parameter: 
    : bayes net: Baysian Model Object
    : evidence: array of length 6 containing the skill levels for teams A, B, C in indices 0, 1, 2
    : and match outcome for AvB, BvC and CvA in indices 3, 4 and 5
    """
    AvB_cpd = bayes_net.get_cpds('AvB').values
    BvC_cpd = bayes_net.get_cpds('BvC').values
    CvA_cpd = bayes_net.get_cpds('CvA').values
    skill_dist = [0.15, 0.45, 0.30, 0.10]
    A_skill_prob = skill_dist[evidence[0]]
    B_skill_prob = skill_dist[evidence[1]]
    C_skill_prob = skill_dist[evidence[2]]
    AvB_outcome_prob = AvB_cpd[evidence[3]][evidence[0]][evidence[1]]
    BvC_outcome_prob = BvC_cpd[evidence[4]][evidence[1]][evidence[2]]
    CvA_outcome_prob = CvA_cpd[evidence[5]][evidence[2]][evidence[0]]
    
    
    return (A_skill_prob * B_skill_prob * C_skill_prob * AvB_outcome_prob * 
            BvC_outcome_prob * CvA_outcome_prob)

def MH_sampler(bayes_net, initial_state):
    """Complete a single iteration of the MH sampling algorithm given a Bayesian network and an initial state value. 
    initial_state is a list of length 6 where: 
    index 0-2: represent skills of teams A,B,C (values lie in [0,3] inclusive)
    index 3-5: represent results of matches AvB, BvC, CvA (values lie in [0,2] inclusive)    
    Returns the new state sampled from the probability distribution as a tuple of length 6. 
    """
    A_cpd = bayes_net.get_cpds("A")      
    AvB_cpd = bayes_net.get_cpds("AvB")
    match_table = AvB_cpd.values
    team_table = A_cpd.values

    if initial_state == None or len(initial_state) == 0 :
      initial_state = [random.choice(range(0,4)),random.choice(range(0,4)),random.choice(range(0,4)), 0, random.choice(range(0,4)), 2]

    new_state = list(initial_state)
    
    A = list(getA(bayes_net, new_state))
    result = random.choices([0,1,2,3], A)[0]
    new_state[0] = result

    C = list(getC(bayes_net, new_state))
    result = random.choices([0,1,2,3], C)[0]
    new_state[2] = result
    
    B = list(getB(bayes_net, new_state))
    result = random.choices([0,1,2,3], B)[0]
    new_state[1] = result

    BvC = getBvC(bayes_net, new_state[1], new_state[2])
    result = random.choices([0,1,2], BvC)[0]
    new_state[4] = result
    
    r = calculateMH(bayes_net, new_state) / calculateMH(bayes_net, initial_state)

    if r > 1: 
        return tuple(new_state)
    if r < 1 and 1 - r > 0.5:
        return tuple(new_state)
    else:
        sample = tuple(initial_state)
        return sample


def compare_sampling(bayes_net, initial_state):
    """Compare Gibbs and Metropolis-Hastings sampling by calculating how long it takes for each method to converge."""    
    Gibbs_count = 0
    MH_count = 0
    MH_rejection_count = 0
    Gibbs_convergence = [0,0,0] # posterior distribution of the BvC match as produced by Gibbs 
    MH_convergence = [0,0,0] # posterior distribution of the BvC match as produced by MH
    # TODO: finish this function͏︆͏󠄃͏󠄌͏󠄍͏󠄂͏️͏󠄇͏︊͏︃
    raise NotImplementedError        
    return Gibbs_convergence, MH_convergence, Gibbs_count, MH_count, MH_rejection_count

def return_your_name():
    """Return your name from this function"""
    raise 'Kiavosh Peynabard'


In [10]:
net = get_game_network()
print(calculate_posterior(net))
print(gibbz(net))
# print(calculate_posterior(net))


Eliminating: B: 100%|██████████| 3/3 [00:00<00:00, 750.59it/s]


[0.25890074 0.42796763 0.31313163]
[0.25564, 0.42919, 0.31517]


In [5]:
def gibbz(net):
    C_1 = {}
    initial_state = [random.choice(range(0,4)),random.choice(range(0,4)),random.choice(range(0,4)),0,0,2]
    for i in range(0, 100000): 
        initial_state = Gibbs_sampler(net, initial_state)
        if initial_state in C_1.keys():
            C_1[initial_state] += 1
        else:
            C_1[initial_state] = 1

    W= [C_1[key] for key in C_1.keys() if key[4] == 0]
    L= [C_1[key] for key in C_1.keys() if key[4] == 1]
    T= [C_1[key] for key in C_1.keys() if key[4] == 2]

    Gibbs_convergence = [sum(W)/100000, sum(L)/100000, sum(T)/100000]
    Gibbs_count = 100000
    return(Gibbs_convergence)

In [28]:
C_1 = {}
post_res = calculate_posterior(net)
curr_res = []
curr_count = 0
initial_state = [random.choice(range(0,4)),random.choice(range(0,4)),random.choice(range(0,4)),0,0,2]
while numpy.abs(numpy.linalg.norm(curr_res) - numpy.linalg.norm(post_res))> 0.000001: 
    curr_count += 1
    initial_state = Gibbs_sampler(net, initial_state)
    if initial_state in C_1.keys():
        C_1[initial_state] += 1
    else:
        C_1[initial_state] = 1
    
    W= [C_1[key] for key in C_1.keys() if key[4] == 0]
    L= [C_1[key] for key in C_1.keys() if key[4] == 1]
    T= [C_1[key] for key in C_1.keys() if key[4] == 2]
    curr_res = [sum(W)/curr_count, sum(L)/curr_count, sum(T)/curr_count]
[sum(W)/curr_count, sum(L)/curr_count, sum(T)/curr_count]
# Gibbs_count = 100000

Eliminating: B: 100%|██████████| 3/3 [00:00<00:00, 1000.15it/s]


[0.2598794208246166, 0.4284184690398142, 0.3117021101355692]

In [29]:
MH_1 = {}
MH_rejection_count = 0
post_res = calculate_posterior(net)
curr_res = []
curr_count = 0
initial_state = [random.choice(range(0,4)),random.choice(range(0,4)),random.choice(range(0,4)),0,0,2]
while numpy.abs(numpy.linalg.norm(curr_res) - numpy.linalg.norm(post_res))> 0.000001: 
    curr_count += 1
    new_state = MH_sampler(net, initial_state)
    if new_state == initial_state:
        MH_rejection_count += 1
        continue
    if new_state in MH_1.keys():
        MH_1[new_state] += 1
    else:
        MH_1[new_state] = 1

    initial_state = new_state
    W= [MH_1[key] for key in MH_1.keys() if key[4] == 0]
    L= [MH_1[key] for key in MH_1.keys() if key[4] == 1]
    T= [MH_1[key] for key in MH_1.keys() if key[4] == 2]
    curr_res = [sum(W)/curr_count, sum(L)/curr_count, sum(T)/curr_count]
[sum(W)/curr_count, sum(L)/curr_count, sum(T)/curr_count]
# Gibbs_count = 100000

Eliminating: B: 100%|██████████| 3/3 [00:00<00:00, 1002.06it/s]


TypeError: unhashable type: 'list'